# Dynamic optimization

## Intro

In [1]:
# TODO: description and flow chart

## Init

In [2]:
# imports
import os #to access system folders
import subprocess #to access ffmpeg in the system
import numpy as np #easy vector operations
import math #for operation with infinite
from scipy.optimize import curve_fit #fittin of the curve
import json #to handle json files
import matplotlib.pyplot as pl #to display plots

#constants
PARAM_AVC = {"crfs": 52, "starting_range": [17,28], "lib": "libx264", "container": "mp4", "add_param": ""}
PARAM_HEVC = {"crfs": 52, "starting_range": [22,33], "lib": "libx265", "container": "mp4", "add_param": ""}
PARAM_VP9 = {"crfs": 64, "starting_range": [15,35], "lib": "libvpx-vp9", "container": "webm", "add_param": "-b:v 0"}

#variables
input_type = "yuv" #valules:"yuv", "y4m", "seq"
codec = "avc" #values: "avc", "hevc", "vp9", "av1", "vvc"

raw_width = 1920 #opz. only for yuv and seq
raw_height = 1080 #opz. only for yuv and seq
raw_fps = 29.97 #opz. only for yuv and seq

source_name = "testscene01"
source_path = "test_vids/srcRAW_FullHD/" + source_name + "." + input_type
ref_path = "test_vids/tempRAW_refs/" #raw files for each shot
dist_path = "test_vids/temp_encoded/" #encoded files for each shot
#assessment files path
tm_file = "rd_results/template.json"
rd_file = "rd_results/" + source_name + ".json"
vmaf_logs = "rd_results/vmaf_logs"

current_point = None #the current optimum point
new_point = 0 #new value to compare with current_point
#TODO: compare with everything stored
res_matrix = {"crf": None, "bitrate": None, "vmaf": None, "psnr": None} #all computed points, by row: crf, bitrate, vmaf, psnr

flag_target = True #values True (quality) or False (bitrate)
quality_metric = "vmaf" #values "vmaf", "psnr", "ssim", "mssim"
#TODO implement more quality metrics
target_bitrate = [12000000]
target_quality = [96]

print("init done")

init done


## Input and shot change detection

In [3]:
def init_res_matrix(x):
    res_matrix["crf"] = np.arange(0,x,1).tolist()
    inf_matrix = np.zeros(x) #infinity to avoid considering zero as a point
    inf_matrix[inf_matrix == 0] = math.inf
    res_matrix["bitrate"] = inf_matrix.tolist()
    res_matrix["vmaf"] = inf_matrix.tolist()
    res_matrix["psnr"] = inf_matrix.tolist()

An empty json structure is generated to be filled and to store computed values

In [4]:
struct_points = [] #structure of target points for json file
struct_shots = [] #structure of shots for json file
num_scenes = 3 #TODO: shot change detection

#TODO: support different input raw files
if input_type == "yuv":
    print("yuv input")
elif input_type == "y4m":
    print("y4m input")
else:
    print("No such an input type")
    exit()
    
#TODO: create folers inside tempENCODED with indexes
    
#init values based on the selected output codec
if codec == "avc":
    s_cod = PARAM_AVC
    init_res_matrix(PARAM_AVC["crfs"])
elif codec == "hevc":
    s_cod = PARAM_HEVC
    init_res_matrix(PARAM_HEVC["crfs"])
elif codec == "vp9":
    s_cod = PARAM_VP9
    init_res_matrix(PARAM_VP9["crfs"])
else:
    print("No such an codec")
    exit()

min_range_crf = s_cod["starting_range"][0]
max_range_crf = s_cod["starting_range"][1]
    
#if not os.path.isfile(rd_file):
    #TODO: duplicate file and rename
with open(tm_file, 'r') as f:
    o_data = json.load(f)
    
    #add source name and results matrix
    o_data["content"] = source_name
    o_data["versions"][0]["codec"] = codec
    o_data["versions"][0]["width"] = raw_width
    o_data["versions"][0]["height"] = raw_height
    o_data["versions"][0]["fps"] = raw_fps
    o_data["versions"][0]["shots"][0]["assessment"] = res_matrix
    
    #add emplty target points
    base_point = o_data["versions"][0]["shots"][0]["opt_points"][0]
    y = lambda x: target_quality if x else target_bitrate
    for i in range(0, len(y(flag_target))):
        base_point["target"] = y(flag_target)[i]
        struct_points.append(base_point.copy())
    o_data["versions"][0]["shots"][0]["opt_points"] = struct_points
    
    #add empty shots
    base_shot = o_data["versions"][0]["shots"][0]
    for i in range(0, num_scenes):
        base_shot["index"] = i #assign index to shots in json file
        struct_shots.append(base_shot.copy())
    o_data["versions"][0]["shots"] = struct_shots

with open(rd_file, 'w') as w:
    json.dump(o_data, w, separators=(',',': '))

yuv input


## Optimization
Find the shot encoded to a certain crf that has the closest quality or rate to the target

In [5]:
#store the quality and rate results for each shot at each encoded crf
def save_results(index, crf, bitrate, vmaf, psnr):
    with open(rd_file, 'r') as f:
        o_data = json.load(f)
        o_data["versions"][0]["shots"][index]["assessment"]["crf"][crf] = crf
        o_data["versions"][0]["shots"][index]["assessment"]["bitrate"][crf] = bitrate
        o_data["versions"][0]["shots"][index]["assessment"]["vmaf"][crf] = vmaf
        o_data["versions"][0]["shots"][index]["assessment"]["psnr"][crf] = psnr
    with open(rd_file, 'w') as w:
        json.dump(o_data, w, separators=(',',': '))
    print("values for -shot" + str(index) + " -crf" + str(crf) + " saved")
    print("-rate" + str(bitrate) + " -vmaf" + str(vmaf))
    return o_data["versions"][0]["shots"][index]["assessment"]

#linear interpolation of the target and the weight alpha between sx and dx
def interpolate(mat, sx, dx):
    alpha = (mat[target_name][sx] - target) / (mat[target_name][sx] - mat[target_name][dx])
    new_point = round(mat["crf"][sx] - alpha * (mat["crf"][sx] - mat["crf"][dx]))
    print("new -crf" + str(new_point))
    return new_point

In [6]:
shot_index = 0
point_index = 0

for shot in sorted(os.listdir(ref_path)): #for each shot
    print("init computing -scene" + str(shot_index))
    while not current_point == new_point: #if no convergence
        if point_index == 0: #if there are no points to compare
            new_point = max_range_crf #encode at the upper value iof the starting range
            
        #encoding
        add_info = s_cod["add_param"]
        lib = s_cod["lib"]
        out = dist_path + str(shot_index) + "/" + str(new_point) + "_" + codec.upper() + "." + s_cod["container"]
        enc = f"ffmpeg -f rawvideo -video_size {raw_width}x{raw_height} \
            -r {raw_fps} -pixel_format yuv420p -i {ref_path+shot} -c:v {lib} \
            -crf {new_point} {add_info} {out}"
        subprocess.call(enc, shell=True)
        
        #quality assessment
        c_vmaf = f"ffmpeg -f rawvideo -r {raw_fps} -video_size {raw_width}x{raw_height} -i {ref_path+shot} \
            -i {out} \
            -lavfi \"[0:v]setpts=PTS-STARTPTS[ref];\
                     [1:v]scale={raw_width}x{raw_height}:flags=bicubic, setpts=PTS-STARTPTS[dist];\
                     [dist][ref]libvmaf=feature=name=psnr:log_path={vmaf_logs}:log_fmt=json\" \
            -f null -" #|name=float_ssim|name=float_ms_ssim to compute the other metrics
        subprocess.call(c_vmaf, shell=True)
        
        #extract quality and rate values
        with open(vmaf_logs, 'r') as r:
            i_data = json.load(r)
        vmaf = i_data["pooled_metrics"]["vmaf"]["mean"]
        psnr = (6*i_data["pooled_metrics"]["psnr_y"]["mean"] + i_data["pooled_metrics"]["psnr_cb"]["mean"] + i_data["pooled_metrics"]["psnr_cr"]["mean"])/8
        info = "ffprobe -v error -select_streams v:0 -show_entries format:stream -print_format json " + out
        cout = subprocess.run(info.split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT).stdout
        dict = json.loads(cout)
        bitrate = int(dict['format']['bit_rate'])
    
        #TODO: results must be weighted based on duration
        res_matrix = save_results(shot_index, new_point, bitrate, vmaf, psnr)
        
        if flag_target:
            target_name = "vmaf"
            target = target_quality[0] #TODO: support more targets
        else:
            target_name = "bitrate"
            target = target_bitrate[0]
            
        if point_index == 0: #if there are no points to compare (first loop)
            current_point = max_range_crf #the current optimal point is the first one
            new_point = min_range_crf #in the next loop encode at the second crf (the lower in the starting range)
        else:
            #element-wise difference between the metric and its target value
            difference = np.asarray(abs(np.asarray(res_matrix[target_name]) - target))
            #the minimum difference = the element with the index closer to the target
            i_first_min = np.argmin(difference)
            nd_diff = difference.copy()
            nd_diff[i_first_min] = np.inf #replace the minimum with inf
            i_second_min = np.argmin(nd_diff) #find the second minimum
            current_point = i_first_min #the index of the point closer to the target
            #swap the values of the two ends if the lower end is bigger than the upper end
            if(res_matrix[target_name][i_first_min] > res_matrix[target_name][i_second_min]):
                sx_end = res_matrix[target_name][i_first_min]
                dx_end = res_matrix[target_name][i_second_min]
                i_sx_end = i_first_min
                i_dx_end = i_second_min
            else:
                sx_end = res_matrix[target_name][i_second_min]
                dx_end = res_matrix[target_name][i_first_min]
                i_sx_end = i_second_min
                i_dx_end = i_first_min
            
            if target < sx_end and target > dx_end: #if the target is in the range
                new_point = interpolate(res_matrix, i_sx_end, i_dx_end)
            else: #if the target is out of the range
                if target > sx_end: #if the target is out of the range in the left side
                    new_point = 0 #if no other points in this direction had been stored encode at the min crf
                    i = i_sx_end - 1
                    while new_point == 0 or not i == 0:
                        #the first point you find is the new lower end of the range
                        if not res_matrix[target_name][i] == math.inf:
                            new_point = interpolate(res_matrix, i, i_sx_end)
                        i -= 1
                else: #if the target is out of the range in the right side
                    new_point = s_cod["crfs"] #if no other points in this direction had been stored encode at the max crf
                    i = i_dx_end + 1
                    while new_point == 0 or not i == s_cod["crfs"]:
                        #the first point you find is the new upper end of the range
                        if not res_matrix[target_name][i] == math.inf:
                            new_point = interpolate(res_matrix, i_dx_end, i)
                        i += 1
        point_index += 1
    
    print("Opt point: " + str(current_point))
    current_point = None
    new_point = 0    
    shot_index += 1
    point_index = 0

init computing -scene0


ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable-libopus --enable-libsvtav1 --enable-libdav1d --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libvmaf --enable-version3 --enable-nonfree
  libavutil      57. 24.101 / 57. 24.101
  libavcodec     59. 26.100 / 59. 26.100
  libavformat    59. 22.100 / 59. 22.100
  libavdevice    59.  6.100 / 59.  6.100
  libavfilter     8. 33.100 /  8. 33.100
  libswscale      6.  6.100 /  6.  6.100
  libswresample   4.  6.100 /  4.  6.100
  libpostproc    56.  5.100 / 56.  5.100

values for -shot0 -crf28 saved
-rate5777861 -vmaf84.699862


frame=   60 fps= 22 q=-1.0 Lsize=    9357kB time=00:00:01.90 bitrate=40302.9kbits/s speed=0.699x    x    
video:9356kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.017661%
[libx264 @ 0x55d2b1534180] frame I:1     Avg QP:20.03  size:340136
[libx264 @ 0x55d2b1534180] frame P:15    Avg QP:20.60  size:272704
[libx264 @ 0x55d2b1534180] frame B:44    Avg QP:23.23  size:117019
[libx264 @ 0x55d2b1534180] consecutive B-frames:  1.7%  0.0%  5.0% 93.3%
[libx264 @ 0x55d2b1534180] mb I  I16..4:  0.1% 99.4%  0.5%
[libx264 @ 0x55d2b1534180] mb P  I16..4:  0.0% 40.1%  1.4%  P16..4: 14.0% 22.8% 21.7%  0.0%  0.0%    skip: 0.0%
[libx264 @ 0x55d2b1534180] mb B  I16..4:  0.0%  7.6%  0.3%  B16..8: 24.9% 18.7%  8.6%  direct:23.8%  skip:16.2%  L0:27.6% L1:21.9% BI:50.5%
[libx264 @ 0x55d2b1534180] 8x8 transform intra:96.7% inter:79.7%
[libx264 @ 0x55d2b1534180] coded y,uvDC,uvAC intra: 98.4% 94.1% 64.3% inter: 70.4% 60.8% 12.0%
[libx264 @ 0x55d2b1534180] i16 v,h,dc,p:  7% 28% 

values for -shot0 -crf17 saved
-rate38270314 -vmaf98.361118
new -crf19


...

frame=   60 fps= 50 q=-1.0 Lsize=     600kB time=00:00:01.90 bitrate=2585.6kbits/s speed=1.59x     
video:599kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.256382%
[libx264 @ 0x55a6ae64d180] frame I:1     Avg QP:29.47  size:120810
[libx264 @ 0x55a6ae64d180] frame P:15    Avg QP:30.17  size: 17764
[libx264 @ 0x55a6ae64d180] frame B:44    Avg QP:33.63  size:  5118
[libx264 @ 0x55a6ae64d180] consecutive B-frames:  1.7%  0.0%  5.0% 93.3%
[libx264 @ 0x55a6ae64d180] mb I  I16..4:  1.3% 86.1% 12.5%
[libx264 @ 0x55a6ae64d180] mb P  I16..4:  0.0%  1.1%  0.3%  P16..4: 29.4%  6.8%  4.4%  0.0%  0.0%    skip:57.8%
[libx264 @ 0x55a6ae64d180] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8: 42.0%  1.5%  0.2%  direct: 0.3%  skip:55.9%  L0:46.8% L1:50.8% BI: 2.3%
[libx264 @ 0x55a6ae64d180] 8x8 transform intra:84.2% inter:82.9%
[libx264 @ 0x55a6ae64d180] coded y,uvDC,uvAC intra: 89.4% 42.3% 13.0% inter: 5.3% 2.4% 0.1%
[libx264 @ 0x55a6ae64d180] i16 v,h,dc,p: 40% 12%  5% 43%
[l

values for -shot2 -crf28 saved
-rate2455197 -vmaf84.94943


frame=   60 fps= 20 q=-1.0 Lsize=   10653kB time=00:00:01.90 bitrate=45884.9kbits/s speed=0.627x    
video:10652kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.015476%
[libx264 @ 0x56437a410180] frame I:1     Avg QP:18.47  size:409737
[libx264 @ 0x56437a410180] frame P:15    Avg QP:18.82  size:301184
[libx264 @ 0x56437a410180] frame B:44    Avg QP:22.05  size:135890
[libx264 @ 0x56437a410180] consecutive B-frames:  1.7%  0.0%  5.0% 93.3%
[libx264 @ 0x56437a410180] mb I  I16..4:  0.0% 99.2%  0.8%
[libx264 @ 0x56437a410180] mb P  I16..4:  0.0% 23.6%  0.7%  P16..4: 24.0% 30.8% 20.9%  0.0%  0.0%    skip: 0.0%
[libx264 @ 0x56437a410180] mb B  I16..4:  0.0%  0.8%  0.0%  B16..8: 37.9% 14.6%  5.4%  direct:30.8%  skip:10.3%  L0:27.2% L1:23.8% BI:49.0%
[libx264 @ 0x56437a410180] 8x8 transform intra:97.5% inter:78.0%
[libx264 @ 0x56437a410180] coded y,uvDC,uvAC intra: 99.3% 96.8% 68.6% inter: 79.0% 58.0% 3.0%
[libx264 @ 0x56437a410180] i16 v,h,dc,p: 18%  0%  6% 7

values for -shot2 -crf17 saved
-rate43570827 -vmaf96.458465
new -crf17
Opt point: 17


frame=   60 fps=9.2 q=-0.0 Lsize=N/A time=00:00:02.00 bitrate=N/A speed=0.308x    
video:28kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
[Parsed_libvmaf_3 @ 0x5652a53cb100] VMAF score: 96.458465


## TODO: Encode opt video
Put together all the individually encoded shots

## TODO: Curve fitting
When the upper search has tested 3 points, given these 3 RQ points, discover the polynomian or logarithmic function that describes their trend. Repeat this when a new point is computed. Measure the error between the approximation and the actual implementation (lagrangian search above) and assess whether and when it may be useful to speed up the search process, by reducing the number of test to encode before the optimum.